# 셀레니움을 사용해서 서울, 경기지역 스타벅스 정보 추출

### 모듈 셋업

In [1]:
# 기본셋팅 - 사용할 모둘 import

In [2]:
# 데이터 구조를 위하여
import pandas as pd
import numpy as np

# 웹 크롤링을 위하여
from bs4 import BeautifulSoup
from selenium import webdriver

# 크롤링 통신을 하기 위해서
import requests
import json
from pandas.io.json import json_normalize

# webdriver의 Select와 Key를 위하여
from selenium.webdriver.support.ui import Select  # Select
from selenium.webdriver.common.keys import Keys   # Key

# 기타
import time

### 데이터 추출

In [3]:
# 웹 드라이버 실행
driver = webdriver.Chrome("chromedriver")

In [4]:
# Click function :

def click(area):
    # 페이지 입장
    driver.get("https://www.starbucks.co.kr/store/store_map.do?disp=locale")
    time.sleep(5)
    driver.implicitly_wait(30)    # implicitly 함수 : 기다리는 동안 값을 가져오는데 필요한 시간??
    
    # 각 지정 지역 클립
    ss = driver.find_element_by_class_name('sido_arae_box')
    s1 = ss.find_elements_by_tag_name('li')
    time.sleep(3)
    driver.implicitly_wait(10)
    s1[area].click()
    
    ############## 로딩 대기 ##############
    time.sleep(3)                                      # 동적요소에서 무조건 기다리는 의미
    driver.implicitly_wait(30)                         # 클릭해서 기다리는 시간?
    
    ############### 전체 클릭 ##############
    gg = driver.find_element_by_class_name('gugun_arae_box')
    g1 = gg.find_elements_by_tag_name('li')                    # 여러개를 불러올때는 s (복수)로 적어주자
    time.sleep(2)
    driver.implicitly_wait(10)
    g1[0].click()
    
    ############### 로딩 대기 ##############
    time.sleep(5)
    driver.implicitly_wait(30)
    
    # 소스 코드
    source = driver.page_source
    bs = BeautifulSoup(source, 'lxml')
    return bs      

In [5]:
def getInfo(bs):
    # 매장정보 접근
    ul = bs.find('ul', class_ = 'quickSearchResultBoxSidoGugun')
    sb_list = ul.find_all('li')
    
    time.sleep(3)
    driver.implicitly_wait(30)
    
    # 매장의 정보를 담을 리스트 생성
    names = []
    lat = []
    long = []
    add = []
    
    # 각 매장의 정보를 받아온다
    for sb in sb_list:
        names.append(sb['data-name'])
        lat.append(sb['data-lat'])
        long.append(sb['data-long'])
        add.append(sb.find('p').text)
        
    # 데이터프레임 생성 
    area = pd.DataFrame([names, lat, long, add], index=['매장이름', '위도', '경도', '주소']).T   #.T는 치환의 의미로 사용
    return area

In [6]:
# 실행
# 서울 = 0, 경기 = 1

bs = click(0)
ss_sb = getInfo(bs)
bs = click(1)
gg_sb = getInfo(bs)

<ipython-input-4-24ba458d7145>:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ss = driver.find_element_by_class_name('sido_arae_box')
C:\Users\user\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:358: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
<ipython-input-4-24ba458d7145>:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  gg = driver.find_element_by_class_name('gugun_arae_box')


In [7]:
# 검색된 수를 확인
len(ss_sb), len(gg_sb)

(567, 362)

In [10]:
# head는 data의 10개 행만 앞단에 출력이 된다 / head.tail을 사용하면 끝에서 10가지를 출력해준다.
gg_sb.head(10)  

,매장이름,위도,경도,주소
0,남이섬,37.806624,127.52527,경기도 가평군 가평읍 북한강변로 1054 북한강변로 10541522-3232
1,소노캄고양,37.6618,126.7507,경기도 고양시 일산동구 장항동 1755 1522-3232
2,일산애니골DT,37.6748,126.793181,경기도 고양시 일산동구 무궁화로 294 (풍동)1522-3232
3,동국대일산병원,37.676411,126.806405,경기도 고양시 일산동구 식사동 814 경기도 고양시 일산동구 동국로 271522-3232
4,일산풍동DT,37.662091,126.800900,경기도 고양시 일산동구 백마로 475 (풍동)1522-3232
5,백석중앙로,37.641100,126.790987,경기도 고양시 일산동구 강송로 33 (백석동)1522-3232
6,일산식사,37.6779381,126.812161,경기도 고양시 일산동구 위시티2로11번길 31 (식사동)1522-3232
7,풍산이마트,37.673769,126.786883,경기도 고양시 일산동구 무궁화로 237 (중산동)1522-3232
8,백석역DT,37.6439377,126.7873856,"경기도 고양시 일산동구 중앙로 1054, 1층 (백석동)1522-3232"
9,백석,37.643888,126.786153,경기도 고양시 일산동구 중앙로 1059 (백석동)1522-3232


In [11]:
ss_sb.head(10)

,매장이름,위도,경도,주소
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동)1522-3232
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동)1522-3232
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동)1522-3232
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동)1522-3232
4,스탈릿대치R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동)1522-3232
5,봉은사역,37.515000,127.063196,서울특별시 강남구 봉은사로 619 (삼성동)1522-3232
6,압구정윤성빌딩,37.5227934,127.0286009,서울특별시 강남구 논현로 834 (신사동)1522-3232
7,코엑스별마당,37.510150,127.060275,서울특별시 강남구 영동대로 513 (삼성동)1522-3232
8,삼성역섬유센터R,37.507750,127.060651,서울특별시 강남구 테헤란로 518 (대치동)1522-3232
9,압구정R,37.5273669,127.033061,서울특별시 강남구 언주로 861 (신사동)1522-3232
